In [1]:
import datetime as dt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
today = dt.date.today()
month, day, year = today.strftime("%m"), today.strftime("%d"), today.strftime("%Y")

In [3]:
### input date ###
### returns dataframe ###

def get_days_games(year, month, day):
# convert date into strings if needed
    year, month, day = str(year), str(month), str(day)
    
    url = 'https://www.covers.com/sports/ncaab/matchups?selectedDate='
    url = url+year+"-"+month+"-"+day
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    games = soup.find_all("div", class_="cmg_matchup_game_box cmg_game_data")
    visitor = []
    home = []
    date = []
    line = []
    for game in games:
        visitor.append(game["data-away-team-fullname-search"])
        home.append(game["data-home-team-fullname-search"])
        date.append(game["data-game-date"])
        line.append(game["data-game-odd"])
    return pd.DataFrame(zip(date, home, visitor, line), columns = ["Date", "Home", "Away", "Line"])

In [4]:
def get_range_of_days_games(year, month, day, year_f, month_f, day_f):

# initialize variables    
    current_date = dt.date(year, month, day)
    visitor, home, date, line = [], [], [], []
    master_df = pd.DataFrame()
                            
    while(current_date != dt.date(year_f, month_f, day_f)):
        
# reset lists
        visitor, home, date, line = [], [], [], []
# get todays lines
        df = get_days_games(year, month, day)
#combine with master
        master_df = pd.concat([master_df, df])                
# increment day, month, year
        current_date += dt.timedelta(days=1)
        day = current_date.day
        month = current_date.month
        year = current_date.year
    return master_df


In [5]:
df = get_range_of_days_games(2020,11, 25, 2021, 3,7)

In [6]:
df = df[df.duplicated()==False]

In [7]:
df[df['Away']=="Michigan"]

,Date,Home,Away,Line
2,2020-12-25 18:00:00,Nebraska,Michigan,7.5
11,2020-12-31 20:00:00,Maryland,Michigan,2
125,2021-01-09 14:15:00,Penn St.,Michigan,
17,2021-01-16 14:00:00,Minnesota,Michigan,5.5
9,2021-01-22 19:00:00,Purdue,Michigan,5
40,2021-01-27 19:00:00,Penn St.,Michigan,
32,2021-02-03 21:00:00,Northwestern,Michigan,
4,2021-02-14 13:00:00,Wisconsin,Michigan,0
6,2021-02-21 13:00:00,Ohio St.,Michigan,1.5
6,2021-02-27 12:00:00,Indiana,Michigan,


In [8]:
df.to_csv("2020_2021_lines.csv")